In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_strategy_returns(df, transaction_cost=0.006):
    """
    Calculate the returns of a trading strategy, including the effect of transaction costs.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Price' and 'predicted label' columns.
    transaction_cost (float): The transaction cost per trade as a percentage. Default is 0.006 (0.6%).

    Returns:
    pd.DataFrame: Updated DataFrame with additional columns for percentage change, strategy returns,
                  transaction costs, and strategy returns after transaction costs.
    """
    # Calculate the percentage change in price from one row to the next
    df['Pct_Change'] = df['Price'].pct_change().fillna(0)
    
    # Calculate the strategy return by multiplying percentage change by the predicted label
    df['Strategy_Return'] = df['Pct_Change'] * df['predicted label']
    
    # Calculate the transaction cost based on the absolute value of the predicted label
    df['Transaction_Cost'] = df['predicted label'].abs() * transaction_cost
    
    # Calculate the strategy return after transaction costs
    df['Strategy_Return_TC'] = df['Strategy_Return'] - df['Transaction_Cost']
    
    # Set strategy returns to 0 where the predicted label is 0 (no position)
    df.loc[df['predicted label'] == 0, ['Strategy_Return', 'Strategy_Return_TC']] = 0
    
    return df

In [3]:
def calculate_overall_return(df, column):
    """
    Calculate the overall return for a given column in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing the returns data.
    column (str): The column name for which the overall return is to be calculated.

    Returns:
    float: The overall return calculated as the cumulative product of (1 + returns) minus 1.
    """
    # Calculate the overall return by computing the cumulative product of (1 + returns) and subtracting 1
    overall_return = (df[column] + 1).prod() - 1
    
    return overall_return

In [4]:
def calculate_sharpe_ratio(df, column, risk_free_rate=0.01):
    """
    Calculate the Sharpe Ratio for a given column in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing the returns data.
    column (str): The column name for which the Sharpe Ratio is to be calculated.
    risk_free_rate (float): The annual risk-free rate, default is 0.01 (1%).

    Returns:
    float: The Sharpe Ratio, which is the ratio of excess return to standard deviation of the returns.
    """
    # Calculate the mean excess return by subtracting the daily risk-free rate from the mean return
    excess_return = df[column].mean() - risk_free_rate / 365
    
    # Calculate the standard deviation of the returns
    std_dev = df[column].std()
    
    # Calculate the Sharpe Ratio by dividing the excess return by the standard deviation
    # If the standard deviation is zero, return 0 to avoid division by zero
    return excess_return / std_dev if std_dev != 0 else 0.0

In [5]:
def calculate_max_drawdown(df, column):
    """
    Calculate the maximum drawdown for a given column in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing the returns data.
    column (str): The column name for which the maximum drawdown is to be calculated.

    Returns:
    float: The maximum drawdown, which is the largest peak-to-trough decline in the cumulative return.
    """
    # Calculate the cumulative return by computing the cumulative product of (1 + returns)
    cumulative_return = (df[column] + 1).cumprod()
    
    # Calculate the cumulative maximum of the cumulative return
    cumulative_max = cumulative_return.cummax()
    
    # Calculate the drawdown by comparing the cumulative return to the cumulative maximum
    drawdown = (cumulative_return - cumulative_max) / cumulative_max
    
    # Return the minimum value of the drawdown series, which represents the maximum drawdown
    return drawdown.min()

In [6]:
def calculate_metrics(df, periods, transaction_cost):
    """
    Calculate evaluation metrics for a given strategy.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the strategy returns.
    - periods (str): Time period for calculating metrics, either 'daily' or 'weekly'.
    - transaction_cost (float): Transaction cost applied to the strategy returns.

    Returns:
    - metrics (dict): Dictionary containing the calculated metrics.
    """
    
    metrics = {}
    for column in ['Strategy_Return', 'Strategy_Return_TC']:
        suffix = '' if column == 'Strategy_Return' else '_TC'
        metrics[f'overall_return{suffix}'] = calculate_overall_return(df, column)      
        metrics[f'sharpe_ratio{suffix}'] = calculate_sharpe_ratio(df, column)
        metrics[f'maximum_drawdown{suffix}'] = calculate_max_drawdown(df, column)
    return metrics

In [7]:
def calculate_metrics_for_series(df, returns, returns_tc, periods, transaction_cost):
    """
    Calculate evaluation metrics for a Buy and Hold strategy.

    Parameters:
    - df (pd.DataFrame): DataFrame containing the strategy data.
    - returns (pd.Series): Series containing the returns without transaction cost.
    - returns_tc (pd.Series): Series containing the returns with transaction cost.
    - periods (str): Time period for calculating metrics, either 'daily' or 'weekly'.
    - transaction_cost (float): Transaction cost applied to the strategy returns.

    Returns:
    - metrics (dict): Dictionary containing the calculated metrics.
    """
    
    metrics = {}
    metrics['overall_return'] = (returns + 1).prod() - 1
    metrics['overall_return_TC'] = (returns_tc + 1).prod() - 1
    
    metrics['sharpe_ratio'] = calculate_sharpe_ratio(pd.DataFrame(returns, columns=['Pct_Change']), 'Pct_Change')
    metrics['sharpe_ratio_TC'] = calculate_sharpe_ratio(pd.DataFrame(returns_tc, columns=['Pct_Change']), 'Pct_Change')
    
    metrics['maximum_drawdown'] = calculate_max_drawdown(pd.DataFrame(returns, columns=['Pct_Change']), 'Pct_Change')
    metrics['maximum_drawdown_TC'] = calculate_max_drawdown(pd.DataFrame(returns_tc, columns=['Pct_Change']), 'Pct_Change')
    
    return metrics

In [8]:
def calculate_profitable_trades(df, transaction_cost):
    """
    Calculate various statistics related to profitable trades in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Pct_Change', 'predicted label', and 'Transaction_Cost' columns.
    transaction_cost (float): The transaction cost per trade as a percentage.

    Returns:
    tuple: A tuple containing the following statistics:
        - Percentage of profitable trades before transaction costs.
        - Percentage of profitable trades after transaction costs.
        - Percentage of trades with a high profit (> 10%).
        - Percentage of trades with a high loss (< -10%).
        - Percentage of trades with a moderate profit (> 5%).
        - Percentage of trades with a moderate loss (< -5%).
        - Total accumulated profit from the strategy.
        - Percentage of profitable trades out of total days.
        - Percentage of profitable trades after transaction costs out of total days.
        - Average return per trade.
        - Average return per period.
        - Median return per trade.
        - Median return per period.
    """
    # Filter trades where the predicted label is not zero
    trades = df[df['predicted label'] != 0]
    total_trades = len(trades)
    total_days = len(df)
    
    # If there are no trades, return zeros for all statistics
    if total_trades == 0:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    
    # Calculate returns adjusted for the predicted label and transaction costs
    adjusted_returns = trades['Pct_Change'] * trades['predicted label']
    adjusted_returns_tc = adjusted_returns - trades['Transaction_Cost']

    # Calculate various statistics
    profitable_trades = adjusted_returns > 0
    profitable_trades_after_tc = adjusted_returns_tc > 0
    high_profit_trades_10 = adjusted_returns > 0.10
    high_loss_trades_10 = adjusted_returns < -0.10
    high_profit_trades_5 = adjusted_returns > 0.05
    high_loss_trades_5 = adjusted_returns < -0.05
    
    pct_profitable_trades = profitable_trades.sum() / total_trades
    pct_profitable_trades_after_tc = profitable_trades_after_tc.sum() / total_trades
    pct_high_profit_trades_10 = high_profit_trades_10.sum() / total_trades
    pct_high_loss_trades_10 = high_loss_trades_10.sum() / total_trades
    pct_high_profit_trades_5 = high_profit_trades_5.sum() / total_trades
    pct_high_loss_trades_5 = high_loss_trades_5.sum() / total_trades
    
    # Calculate total accumulated profit using the calculate_overall_return function
    total_accumulated_profit = calculate_overall_return(trades, 'Strategy_Return')
    
    # Calculate percentage of profitable trades out of total days
    pct_profitable_trades_overall = profitable_trades.sum() / total_days
    pct_profitable_trades_after_tc_overall = profitable_trades_after_tc.sum() / total_days
    
    # Calculate average and median returns per trade and per period
    avg_return_per_trade = adjusted_returns.mean() if total_trades > 0 else 0
    median_return_per_trade = adjusted_returns.median() if total_trades > 0 else 0
    avg_return_per_period = df['Strategy_Return'].mean()
    median_return_per_period = df['Strategy_Return'].median()
    
    # Return all calculated statistics as a tuple
    return (pct_profitable_trades, pct_profitable_trades_after_tc, pct_high_profit_trades_10, pct_high_loss_trades_10, 
            pct_high_profit_trades_5, pct_high_loss_trades_5, total_accumulated_profit, 
            pct_profitable_trades_overall, pct_profitable_trades_after_tc_overall,
            avg_return_per_trade, avg_return_per_period, median_return_per_trade, median_return_per_period)


In [9]:
def evaluate_different_holding_periods(dictionary, holding_periods=[3, 10, 30], initial_capital=10000):
    """
    Evaluate the trading strategy for different holding periods and return a DataFrame summarizing the accumulated returns.

    Parameters:
    dictionary (dict): Nested dictionary containing the data for each labeling method and currency.
    holding_periods (list): List of holding periods to evaluate.
    initial_capital (float): The initial amount of capital available for trading.

    Returns:
    pd.DataFrame: DataFrame summarizing the accumulated returns for each holding period and each labeling method.
    """

    def calculate_strategy_returns_for_holding_period(df, holding_period):
        df['Pct_Change_Holding'] = df['Price'].pct_change(periods=holding_period).shift(-holding_period)
        df['Strategy_Return_Holding'] = df['Pct_Change_Holding'] * df['predicted label']
        df['Strategy_Return_Holding'].fillna(0, inplace=True)
        return df

    def calculate_holding_period_returns(df, holding_period):
        capital = 1.0  # Start with an initial capital of 1
        for start in range(0, len(df), holding_period):
            end = start + holding_period
            if end >= len(df):
                break
            period_return = (df['Price'].iloc[end] / df['Price'].iloc[start] - 1) * df['predicted label'].iloc[start]
            if not pd.isna(period_return):
                capital *= (1 + period_return)
        return capital - 1

    columns = ['labeling_method', 'currency'] + [f'return_{period}_days' for period in holding_periods]
    results_df = pd.DataFrame(columns=columns)

    for labeling_method, currency_data in dictionary.items():
        overall_row = {'labeling_method': labeling_method, 'currency': 'overall'}
        overall_returns = {f'return_{period}_days': 0 for period in holding_periods}
        overall_weights = {f'return_{period}_days': 0 for period in holding_periods}

        for currency, data in currency_data.items():
            row = {'labeling_method': labeling_method, 'currency': currency}

            for holding_period in holding_periods:
                if len(data["test"]) > holding_period:
                    df = calculate_strategy_returns_for_holding_period(data["test"], holding_period)
                    final_cumulative_return = calculate_holding_period_returns(df, holding_period)
                    num_trades = (len(df) - 1) // holding_period

                    row[f'return_{holding_period}_days'] = final_cumulative_return
                    overall_returns[f'return_{holding_period}_days'] += final_cumulative_return * num_trades
                    overall_weights[f'return_{holding_period}_days'] += num_trades
                else:
                    row[f'return_{holding_period}_days'] = 0

            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

        for holding_period in holding_periods:
            if overall_weights[f'return_{holding_period}_days'] > 0:
                overall_row[f'return_{holding_period}_days'] = overall_returns[f'return_{holding_period}_days'] / overall_weights[f'return_{holding_period}_days']
            else:
                overall_row[f'return_{holding_period}_days'] = 0

        results_df = pd.concat([results_df, pd.DataFrame([overall_row])], ignore_index=True)

    return results_df

In [10]:
# Simplified calculate_strategy_returns function without transaction costs
def calculate_strategy_returns_2(df):
    """
    Calculate the returns of a trading strategy without considering transaction costs.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Price' and 'predicted label' columns.

    Returns:
    pd.DataFrame: Updated DataFrame with additional columns for percentage change and strategy returns.
    """
    # Calculate the percentage change in price from one row to the next
    df['Pct_Change'] = df['Price'].pct_change().fillna(0)
    
    # Calculate the strategy return by multiplying percentage change by the predicted label
    df['Strategy_Return'] = df['Pct_Change'] * df['predicted label']
    
    # Set strategy returns to 0 where the predicted label is 0 (no position)
    df.loc[df['predicted label'] == 0, ['Strategy_Return']] = 0
    
    return df

In [11]:
def evaluate_strategy_financially(dictionary, periods, transaction_cost=0.006):
    """
    Evaluate trading strategies and Buy and Hold strategy for various labeling methods and currencies.

    Parameters:
    - dictionary (dict): Nested dictionary containing the data for each labeling method and currency.
    - periods (str): Time period for calculating metrics, either 'daily' or 'weekly'.
    - transaction_cost (float): Transaction cost applied to the strategy returns.

    Returns:
    - dictionary (dict): Updated dictionary with strategy returns.
    - evaluation_df (pd.DataFrame): DataFrame containing the evaluation metrics for each strategy.
    - market_overview_df (pd.DataFrame): DataFrame summarizing the evaluation metrics across all labeling methods.
    """
    
    columns = [
        'labeling_method', 'currency', 'overall_return', 'overall_return_TC',
        'sharpe_ratio', 'sharpe_ratio_TC', 'maximum_drawdown', 'maximum_drawdown_TC'
    ]
    
    evaluation_df = pd.DataFrame(columns=columns)
    market_overview_columns = [
        'labeling_method', 'overall_return', 'overall_return_TC',
        'sharpe_ratio', 'sharpe_ratio_TC', 'maximum_drawdown', 'maximum_drawdown_TC'
    ]
    market_overview_df = pd.DataFrame(columns=market_overview_columns)
    
    # Validate the periods parameter
    if periods not in ["daily", "weekly"]:
        raise ValueError("Invalid value for periods. Use 'daily' or 'weekly'.")
    
    # To store the lengths of the currency data
    currency_lengths = {}
    
    for labeling_method, currency_data in dictionary.items():
        for currency, data in currency_data.items():
            strategy_df = calculate_strategy_returns(data["test"], transaction_cost)
            dictionary[labeling_method][currency]["test"] = strategy_df
            
            metrics = calculate_metrics(strategy_df, periods, transaction_cost)
            
            new_row = pd.DataFrame({
                'labeling_method': labeling_method, 
                'currency': currency, 
                **metrics
            }, index=[0])
            
            evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)
            
            # Calculate the length of the currency data
            if currency not in currency_lengths:
                currency_lengths[currency] = len(strategy_df)
    
    # Buy and Hold Strategy Integration
    processed_currencies = set()
    for currency_data in dictionary.values():
        for currency, data in currency_data.items():
            if currency not in processed_currencies:
                strategy_df = data["test"]
                
                # Calculate buy-and-hold metrics based on the price percentage change
                strategy_df['Pct_Change'] = strategy_df['Price'].pct_change().fillna(0)
                returns_bh = strategy_df['Pct_Change']
                returns_bh_tc = returns_bh - transaction_cost  # Apply daily transaction costs
                
                metrics_bh = calculate_metrics_for_series(strategy_df, returns_bh, returns_bh_tc, periods, transaction_cost)
                
                new_row_bh = pd.DataFrame({
                    'labeling_method': 'Buy and Hold', 
                    'currency': currency, 
                    **metrics_bh
                }, index=[0])
                
                evaluation_df = pd.concat([evaluation_df, new_row_bh], ignore_index=True)
                
                processed_currencies.add(currency)
    
    market_overview_df = calculate_market_overview(evaluation_df, currency_lengths)
    
    return dictionary, evaluation_df, market_overview_df

In [12]:
def calculate_market_overview(evaluation_df, currency_lengths):
    """
    Calculate market overview metrics by averaging evaluation metrics across all labeling methods using weighted averages.

    Parameters:
    - evaluation_df (pd.DataFrame): DataFrame containing the evaluation metrics for each strategy.
    - currency_lengths (dict): Dictionary with currency as keys and their respective lengths as values.

    Returns:
    - market_overview_df (pd.DataFrame): DataFrame summarizing the evaluation metrics across all labeling methods.
    """
    
    metrics = ['overall_return', 'sharpe_ratio', 'maximum_drawdown']
    market_overview_list = []

    for labeling_method in evaluation_df['labeling_method'].unique():
        method_df = evaluation_df[evaluation_df['labeling_method'] == labeling_method]
        method_metrics = {'labeling_method': labeling_method}
        
        for metric in metrics:
            weighted_sum = 0
            weighted_sum_tc = 0
            total_length = 0
            
            for idx, row in method_df.iterrows():
                currency_key = row['currency']
                if currency_key in currency_lengths:
                    currency_length = currency_lengths[currency_key]
                    if not pd.isna(row[metric]) and row[metric] != 0:
                        weighted_sum += row[metric] * currency_length
                    if not pd.isna(row[f'{metric}_TC']) and row[f'{metric}_TC'] != 0:
                        weighted_sum_tc += row[f'{metric}_TC'] * currency_length
                    total_length += currency_length

            if total_length > 0:
                method_metrics[metric] = weighted_sum / total_length
                method_metrics[f'{metric}_TC'] = weighted_sum_tc / total_length
            else:
                method_metrics[metric] = 0
                method_metrics[f'{metric}_TC'] = 0

        market_overview_list.append(method_metrics)
    
    market_overview_df = pd.DataFrame(market_overview_list)
    return market_overview_df

In [13]:
def calculate_profitable_trades(df, transaction_cost):
    """
    Calculate various statistics related to profitable trades in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing 'Pct_Change', 'predicted label', and 'Transaction_Cost' columns.
    transaction_cost (float): The transaction cost per trade as a percentage.

    Returns:
    tuple: A tuple containing the following statistics:
        - Percentage of profitable trades before transaction costs.
        - Percentage of profitable trades after transaction costs.
        - Percentage of trades with a high profit (> 10%).
        - Percentage of trades with a high loss (< -10%).
        - Percentage of trades with a moderate profit (> 5%).
        - Percentage of trades with a moderate loss (< -5%).
        - Total accumulated profit from the strategy.
        - Percentage of profitable trades out of total days.
        - Percentage of profitable trades after transaction costs out of total days.
        - Average return per trade.
        - Average return per period.
        - Median return per trade.
        - Median return per period.
        - Percentage of trades with a profit > 1%.
        - Percentage of trades with a loss < -1%.
        - Percentage of trades with a profit > 0.6%.
        - Percentage of trades with a loss < -0.6%.
    """
    # Filter trades where the predicted label is not zero
    trades = df[df['predicted label'] != 0]
    total_trades = len(trades)
    total_days = len(df)
    
    # If there are no trades, return zeros for all statistics
    if total_trades == 0:
        return 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    
    # Calculate returns adjusted for the predicted label and transaction costs
    adjusted_returns = trades['Pct_Change'] * trades['predicted label']
    adjusted_returns_tc = adjusted_returns - transaction_cost

    # Calculate various statistics
    profitable_trades = adjusted_returns > 0
    profitable_trades_after_tc = adjusted_returns_tc > 0
    high_profit_trades_10 = adjusted_returns > 0.10
    high_loss_trades_10 = adjusted_returns < -0.10
    high_profit_trades_5 = adjusted_returns > 0.05
    high_loss_trades_5 = adjusted_returns < -0.05
    high_profit_trades_1 = adjusted_returns > 0.01
    high_loss_trades_1 = adjusted_returns < -0.01
    high_profit_trades_06 = adjusted_returns > 0.006
    high_loss_trades_06 = adjusted_returns < -0.006
    
    pct_profitable_trades = profitable_trades.sum() / total_trades
    pct_profitable_trades_after_tc = profitable_trades_after_tc.sum() / total_trades
    pct_high_profit_trades_10 = high_profit_trades_10.sum() / total_trades
    pct_high_loss_trades_10 = high_loss_trades_10.sum() / total_trades
    pct_high_profit_trades_5 = high_profit_trades_5.sum() / total_trades
    pct_high_loss_trades_5 = high_loss_trades_5.sum() / total_trades
    pct_high_profit_trades_1 = high_profit_trades_1.sum() / total_trades
    pct_high_loss_trades_1 = high_loss_trades_1.sum() / total_trades
    pct_high_profit_trades_06 = high_profit_trades_06.sum() / total_trades
    pct_high_loss_trades_06 = high_loss_trades_06.sum() / total_trades
    
    # Calculate total accumulated profit using the calculate_overall_return function
    total_accumulated_profit = adjusted_returns.sum()
    
    # Calculate percentage of profitable trades out of total days
    pct_profitable_trades_overall = profitable_trades.sum() / total_days
    pct_profitable_trades_after_tc_overall = profitable_trades_after_tc.sum() / total_days
    
    # Calculate average and median returns per trade and per period
    avg_return_per_trade = adjusted_returns.mean()
    median_return_per_trade = adjusted_returns.median()
    avg_return_per_period = df['Pct_Change'].mean()
    median_return_per_period = df['Pct_Change'].median()
    
    # Return all calculated statistics as a tuple
    return (pct_profitable_trades, pct_profitable_trades_after_tc, pct_high_profit_trades_10, pct_high_loss_trades_10, 
            pct_high_profit_trades_5, pct_high_loss_trades_5, total_accumulated_profit, 
            pct_profitable_trades_overall, pct_profitable_trades_after_tc_overall,
            avg_return_per_trade, avg_return_per_period, median_return_per_trade, median_return_per_period,
            pct_high_profit_trades_1, pct_high_loss_trades_1, pct_high_profit_trades_06, pct_high_loss_trades_06)

In [14]:
def evaluate_trades_overview(dictionary, transaction_cost=0.006):
    """
    Evaluate and summarize the performance of trading strategies for different labeling methods.

    Parameters:
    dictionary (dict): A dictionary where keys are labeling methods and values are dictionaries of currency data.
                       Each currency data dictionary contains DataFrame with trading data under the key "test".
    transaction_cost (float): The transaction cost per trade as a percentage. Default is 0.006 (0.6%).

    Returns:
    pd.DataFrame: A DataFrame summarizing the performance metrics of the trading strategies for each labeling method.
    """
    columns = [
        'Labeling Method', 'Profitable Trades', 'Profitable Trades after TC', 
        'Trades with >10% Profit', 'Trades with >10% Loss',
        'Trades with >5% Profit', 'Trades with >5% Loss', 'Total Accumulated Profit',
        'Percentage of Profitable Trades Total', 'Percentage of Profitable Trades Total after TC',
        'Average Return per Trade', 'Average Return per Period', 'Median Return per Trade', 'Median Return per Period',
        'Trades with >1% Profit', 'Trades with >1% Loss', 'Trades with >0.6% Profit', 'Trades with >0.6% Loss'
    ]
    result_df = pd.DataFrame(columns=columns)
    
    # Iterate over each labeling method and its corresponding currency data
    for labeling_method, currency_data in dictionary.items():
        all_trades_df = pd.DataFrame()
        
        # Iterate over each currency and its data
        for currency, data in currency_data.items():
            # Calculate strategy returns including transaction costs
            strategy_df = data["test"]
            all_trades_df = pd.concat([all_trades_df, strategy_df], ignore_index=True)
        
        # Calculate various trade statistics for the combined trades DataFrame
        (pct_profitable, pct_profitable_tc, pct_high_profit_10, pct_high_loss_10, 
         pct_high_profit_5, pct_high_loss_5, total_accumulated_profit, 
         pct_profitable_overall, pct_profitable_tc_overall, avg_return_per_trade,
         avg_return_per_period, median_return_per_trade, median_return_per_period,
         pct_high_profit_1, pct_high_loss_1, pct_high_profit_06, pct_high_loss_06) = calculate_profitable_trades(all_trades_df, transaction_cost)
        
        # Create a new row with the calculated statistics
        new_row = pd.DataFrame({
            'Labeling Method': labeling_method,
            'Profitable Trades': pct_profitable,
            'Profitable Trades after TC': pct_profitable_tc,
            'Trades with >10% Profit': pct_high_profit_10,
            'Trades with >10% Loss': pct_high_loss_10,
            'Trades with >5% Profit': pct_high_profit_5,
            'Trades with >5% Loss': pct_high_loss_5,
            'Total Accumulated Profit': total_accumulated_profit,
            'Percentage of Profitable Trades Total': pct_profitable_overall,
            'Percentage of Profitable Trades Total after TC': pct_profitable_tc_overall,
            'Average Return per Trade': avg_return_per_trade,
            'Average Return per Period': avg_return_per_period,
            'Median Return per Trade': median_return_per_trade,
            'Median Return per Period': median_return_per_period,
            'Trades with >1% Profit': pct_high_profit_1,
            'Trades with >1% Loss': pct_high_loss_1,
            'Trades with >0.6% Profit': pct_high_profit_06,
            'Trades with >0.6% Loss': pct_high_loss_06
        }, index=[0])
        
        # Append the new row to the result_df DataFrame
        result_df = pd.concat([result_df, new_row], ignore_index=True)
    
    return result_df